## Create External Volume

In [ ]:
CREATE OR REPLACE EXTERNAL VOLUME exvol_vino
   STORAGE_LOCATIONS =
      (
         (
            NAME = 'my-s3-us-west-2'
            STORAGE_PROVIDER = 'S3'
            STORAGE_BASE_URL = 's3://amazon-product-reviews-dataset/'
            STORAGE_AWS_ROLE_ARN = ''
            STORAGE_AWS_EXTERNAL_ID = 'iceberg_table_external_id'
         )
      )
      ALLOW_WRITES = TRUE;

In [ ]:
DESC EXTERNAL VOLUME exvol_vino;

In [ ]:
SELECT SYSTEM$VERIFY_EXTERNAL_VOLUME('exvol_vino');

## Create Catalog Integration

In [ ]:
CREATE OR REPLACE CATALOG INTEGRATION glue_rest_cat_int
  CATALOG_SOURCE = ICEBERG_REST
  TABLE_FORMAT = ICEBERG
  CATALOG_NAMESPACE = 'product_reviews_db'
  REST_CONFIG = (
    CATALOG_URI = 'https://glue.us-west-2.amazonaws.com/iceberg'
    CATALOG_API_TYPE = AWS_GLUE
    CATALOG_NAME = '849350360261'
  )
  REST_AUTHENTICATION = (
    TYPE = SIGV4
    SIGV4_IAM_ROLE = ''
    SIGV4_SIGNING_REGION = 'us-west-2'
  )
  ENABLED = TRUE;

In [ ]:
DESCRIBE CATALOG INTEGRATION glue_rest_cat_int;

In [ ]:
SELECT SYSTEM$VERIFY_CATALOG_INTEGRATION('glue_rest_cat_int');

## Create a Catalog Linked Database (CLD)

In [ ]:
CREATE DATABASE cld_vino_prod
  LINKED_CATALOG = (
    CATALOG = 'glue_rest_cat_int',
    NAMESPACE_MODE = FLATTEN_NESTED_NAMESPACE,
    NAMESPACE_FLATTEN_DELIMITER = '-'
    SYNC_INTERVAL_SECONDS = 60
  )
  EXTERNAL_VOLUME = 'exvol_vino';

In [ ]:
SELECT SYSTEM$CATALOG_LINK_STATUS('cld_vino_prod');

In [ ]:
USE DATABASE "CLD_VINO_PROD";
USE SCHEMA "product_reviews_db";

## Query your Iceberg Tables from Snowflake (through CLD)

In [ ]:
select *
from "product_description_2k";

In [ ]:
select *
from "product_reviews_100k"
limit 100;

## Average Product Ratings by Product Categories

In [ ]:
select B."categories", 
    count(*) as product_volume,
    avg("overall") as avg_rating
from "product_reviews_100k" as A
join "product_description_2k" as B
on A."asin" = B."asin"
group by B."categories"
order by product_volume desc;

## Product Satisfaction Levels by Category 

In [ ]:
WITH category_satisfaction AS (
  SELECT 
    p."categories",
    COUNT(r."reviewerID") as "total_reviews",
    AVG(CAST(r."overall" AS FLOAT)) as "avg_rating",
    COUNT(CASE WHEN CAST(r."overall" AS FLOAT) >= 4.0 THEN 1 END) as "positive_reviews",
    ROUND(
      (COUNT(CASE WHEN CAST(r."overall" AS FLOAT) >= 4.0 THEN 1 END) * 100.0) / 
      COUNT(r."reviewerID"), 2
    ) as "satisfaction_percentage"
  FROM "product_reviews_100k" r
  JOIN "product_description_2k" p ON r."asin" = p."asin"
  WHERE p."categories" IS NOT NULL 
    AND p."categories" != ''
    AND r."overall" IS NOT NULL
  GROUP BY p."categories"
  HAVING COUNT(r."reviewerID") >= 10  -- Only categories with sufficient reviews
)
SELECT 
  "categories",
  "total_reviews",
  ROUND("avg_rating", 2) as "avg_rating",
  "satisfaction_percentage",
  CASE 
    WHEN "satisfaction_percentage" >= 80 THEN 'Excellent'
    WHEN "satisfaction_percentage" >= 70 THEN 'Good'
    WHEN "satisfaction_percentage" >= 60 THEN 'Fair'
    ELSE 'Needs Improvement'
  END as "satisfaction_level"
FROM category_satisfaction
ORDER BY "avg_rating" DESC, "total_reviews" DESC;

## Role Based Access Control: Row Access Policy

In [ ]:
-- Create the sellers_book role
CREATE ROLE sellers_book;

-- Create the row-level access policy for books category access
CREATE ROW ACCESS POLICY books_category_policy AS (categories STRING) RETURNS BOOLEAN ->
  CASE 
    WHEN CURRENT_ROLE() = 'SELLERS_BOOK' THEN 
      categories LIKE '%books%' OR categories LIKE '%book%'
    ELSE 
      FALSE
  END;

-- Apply the policy to the product_description_2k table
ALTER TABLE "product_description_2k" ADD ROW ACCESS POLICY books_category_policy ON (categories);

-- Grant the necessary permissions to the sellers_book role
GRANT USAGE ON DATABASE "cld_vino_prod" TO ROLE sellers_book;
GRANT USAGE ON SCHEMA "product_reviews_db" TO ROLE sellers_book;

GRANT ROLE sellers_book TO USER vinod;
USE ROLE sellers_book;

In [ ]:
select B."categories", 
    count(*) as product_volume,
    avg("overall") as avg_rating
from "product_reviews_100k" as A
join "product_description_2k" as B
on A."asin" = B."asin"
group by B."categories"
order by product_volume desc;

## Setting up Snowflake Intelligence

In [ ]:
use role accountadmin;

GRANT USAGE ON DATABASE cld_amazon_reviews TO ROLE ACCOUNTADMIN;
GRANT USAGE ON SCHEMA cld_amazon_reviews.product_reviews_db TO ROLE ACCOUNTADMIN;

GRANT CREATE AGENT ON SCHEMA cld_amazon_reviews.product_reviews_db TO ROLE ACCOUNTADMIN;

alter user set default_role = ACCOUNTADMIN;
alter user set default_warehouse = vino_m;


GRANT USAGE ON DATABASE snowflake_intelligence TO ROLE ACCOUNTADMIN;
GRANT USAGE ON SCHEMA snowflake_intelligence.agents TO ROLE ACCOUNTADMIN;

GRANT CREATE AGENT ON SCHEMA snowflake_intelligence.agents TO ROLE ACCOUNTADMIN;

alter user set default_role = ACCOUNTADMIN;
alter user set default_warehouse = vino_m;